In [245]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
import yfinance as yf
from IPython.display import Markdown
import re
import datetime
from pykrx.stock import get_market_ticker_list, get_market_ticker_name
from langchain_community.agent_toolkits import FileManagementToolkit
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.document_loaders import PyMuPDFLoader, PyPDFLoader
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.tools import tool
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_core.documents.base import Document
from langchain_core.vectorstores.base import VectorStoreRetriever
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from pydantic import BaseModel, Field
from markitdown import MarkItDown
import dotenv
import os

dotenv.load_dotenv()

True

In [246]:
class State(TypedDict):
    messages : Annotated[list, add_messages]
    df : Annotated[dict, "Stock Dataset"]
    tool_call : Annotated[str, "tool_call Result"]

In [247]:
tools = [TavilySearchResults(), PythonAstREPLTool(), *FileManagementToolkit(
    selected_tools=["read_file", "write_file", "list_directory"]).get_tools(),
]

In [248]:
root_dir="./files"

In [249]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

In [250]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def route(
    state: State,
):
    try:
        message = state["messages"][-1]
    except:
        # 입력 상태에 메시지가 없는 경우 예외 발생
        raise ValueError(f"이전 대화 기록이 존재하지 않습니다.")

    # AI 메시지에 도구 호출이 있는 경우 "tools" 반환
    tool_calls = hasattr(message, "tool_calls")
    if  len(tool_calls) > 0:
        # 도구 호출이 있는 경우 "tools" 반환
        return "tools"
    # 도구 호출이 없는 경우 "END" 반환
    return {"messages":llm.invoke(state["messages"][-1])}

In [252]:
# 주식 DB를 만드는 함수 {"삼성전자":"005930, ... }

def create_stock_db():
    stock_dict = {}
    today = datetime.datetime.today()
    
    stock_list = get_market_ticker_list(today, market="KOSPI")

    for stock in stock_list:
        stock_dict.update({get_market_ticker_name(stock):stock})
    
    stock_list = get_market_ticker_list(today, market="KOSDAQ")

    for stock in stock_list:
        stock_dict.update({get_market_ticker_name(stock):stock})

    return stock_dict

stock_db = create_stock_db()

In [253]:
def search_stock(query):

    """
    주식 검색 도구입니다.
    결과값으로 데이터프레임이 반환됩니다.
    입력 쿼리에서 주식이름을 추출한 후 모든 주식 데이터를 가져옵니다.
    """


    prompt = PromptTemplate.from_template("""
        당신은 주식 이름 추출기입니다.
        주어진 문장에서 주식이름만 추출하세요.

        ### 예시 1
        query : 삼성전자의 최근 1년에 대해서 분석해주세요.

        answer : 삼성전자

        ### 예시 2
        query : AJ홀딩스우의 최근 실적은 얼마인가요?

        answer : AJ홀딩스우

        ### 입력
        query : {query}

        answer : 

        """
        )
    
    chain = prompt | llm | StrOutputParser()

    stock_name = chain.invoke({"query":query})

    try:
        stock_code = stock_db[stock_name.strip().upper()]   
    except:
        raise ValueError(f"종목명 : {stock_name}을/를 검색할 수 없습니다. 오탈자나 한국거래소에서 거래중인 주식인지 확인해주세요.")
    
    # 예: 삼성전자 (한국거래소는 뒤에 '.KS'를 붙임)
    ticker = yf.Ticker(stock_code+".KS")

    df = ticker.history(period="max") # 기간: '1d', '5d', '1mo', '1y', 'max' 등

    return df.reset_index(), stock_name

In [256]:
code_tool = PythonAstREPLTool()

In [ ]:
# tavily_search_results_json, python_repl_ast, read_file, write_file, list_directory

def response(state:State):

    name = state["tool_call"].name

    if name == "tavily_search_results_json":

        result = search_tool.invoke(state["tool_calls"][0]["args"]["query"])
        prompt = ChatPromptTemplate.from_messages(
            ("system", """
                        당신은 아래 내용을 이용하여 답변합니다.
                        생각을 담지 말고 사실만을 전달하세요.
                        내용 : {result}

                        """),
            ("human", "{query}")
        )

        chain = prompt | llm

        answer = chain.invoke({"result":result,
                               "query":state["messages"][-1]}).content
        
        return {"answer": answer}
    
    elif name == "python_repl_ast":

        title = ""

        if len(code_tool.locals) > 0:
            pass
        else:
            try:
                df, title = search_stock(state["query"])
            except:
                pass
            

        code = state["tool_calls"][0]["args"]["query"]
        code_tool.invoke(code)

        prompt = ChatPromptTemplate.from_messages(
            ("system", """
                        아래의 코드를 참고하여 질의에 대해 답변합니다.
                        절대 코드에 대해 설명하지마세요.
                        독자는 프로그래머가 아닙니다.
                        데이터 분석과 관련된 코드가 입력된다면 항상 인사이트를 포함하세요.
                        단순 코드는 단순하게 대답합니다.

                        코드 : {code}
             
                        ### 필요하다면 참고할 것

                        title : {title}

                        """),
            ("human", "{query}")
        )

        chain = prompt | llm

        answer = chain.invoke({"code":code,
                               "title":title,
                               "query":state["messages"][-1]}).content
        
        return {"answer": answer}    

    elif name =="write_file":
        
        root_dir = root_dir
        result = state["tool_calls"][0]["args"]
        
        return write_tool.invoke(result)
    else:
        return {"messages":llm.invoke(state["messages"][-1])}

In [258]:
graph = StateGraph(State)

In [ ]:
graph.add_node("route", route)
graph.add_node("response", response)
graph.add_node("route", route)
graph.add_node("route", route)